In [1]:
import sqlite3
import re

conn = sqlite3.connect("data/recipe.db")
cur = conn.cursor()

def parse_recipes(path):
    with open(path, 'r') as f:
        content = f.read()

    blocks = content.strip().split("\n\n")
    recipes = []

    for block in blocks:
        lines = block.strip().split('\n')
        name = re.match(r"Recipe:\s+(.+)", lines[0]).group(1)
        time = float(re.match(r"Craft Time:\s+([0-9.]+)", lines[1]).group(1))

        ing_start = lines.index("Ingredients:") + 1
        res_start = lines.index("Results:")

        ingredients = []
        for line in lines[ing_start:res_start]:
            if line.strip():
                ing_name, ing_count = line.strip().split()
                ingredients.append((ing_name, int(ing_count)))

        results = []
        for line in lines[res_start+1:]:
            if line.strip():
                res_name, res_count = line.strip().split()
                results.append((res_name, int(res_count)))

        recipes.append({
            "name": name,
            "craft_time": time,
            "ingredients": ingredients,
            "results": results,
        })

    return recipes

def ensure_recipe_exists(name, default_time=0.0):
    cur.execute("SELECT recipe_id FROM Recipes WHERE recipe_name = ?", (name,))
    row = cur.fetchone()
    if row:
        return row[0]
    cur.execute("INSERT INTO Recipes (recipe_name, craft_time) VALUES (?, ?)", (name, default_time))
    return cur.lastrowid

def insert_to_db(recipes):
    for recipe in recipes:
        recipe_id = ensure_recipe_exists(recipe['name'], recipe['craft_time'])

        for ing_name, ing_count in recipe['ingredients']:
            ing_id = ensure_recipe_exists(ing_name)
            cur.execute("""
                INSERT INTO Recipe_Ingredients (recipe_id, ingredient_id, ingredient_count)
                VALUES (?, ?, ?)
            """, (recipe_id, ing_id, ing_count))

        for res_name, res_count in recipe['results']:
            cur.execute("""
                INSERT INTO Recipe_Results (recipe_id, result_name, result_count)
                VALUES (?, ?, ?)
            """, (recipe_id, res_name, res_count))

    conn.commit()

recipes = parse_recipes("data/filtered_recipes.txt")
insert_to_db(recipes)
conn.close()


In [ ]:
import sqlite3
import json
from collections import defaultdict

def pick_display_name(cur, recipe_id, recipe_name_fallback):
    # Prefer the first (arbitrary) result_name, else recipe_name
    cur.execute("""
        SELECT result_name
        FROM Recipe_Results
        WHERE recipe_id = ?
        ORDER BY result_id ASC
        LIMIT 1
    """, (recipe_id,))
    row = cur.fetchone()
    if row and row[0]:
        return row[0]
    return recipe_name_fallback or f"Recipe {recipe_id}"

def export_global_graph(db_path, out_path="recipes.json"):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    # All recipes
    cur.execute("SELECT recipe_id, recipe_name FROM Recipes")
    recipes = cur.fetchall()

    # Nodes for sidebar + global graph (use recipe_id as node id)
    sidebar_nodes = []
    graph_nodes = []
    id_to_name = {}

    for rid, rname in recipes:
        display = pick_display_name(cur, rid, rname)
        sidebar_nodes.append({"recipe_id": rid, "recipe_name": display})
        graph_nodes.append({"id": rid, "name": display})
        id_to_name[rid] = display

    # Edges: ingredient_id -> recipe_id with ingredient_count
    cur.execute("""
        SELECT ri.recipe_id    AS target_recipe_id,
               ri.ingredient_id AS source_recipe_id,
               COALESCE(ri.ingredient_count, 1) AS count
        FROM Recipe_Ingredients ri
    """)
    links = []
    for target_id, source_id, count in cur.fetchall():
        # Guard against orphan references
        if source_id in id_to_name and target_id in id_to_name:
            links.append({
                "source": source_id,
                "target": target_id,
                "count": count
            })

    conn.close()

    data = {
        "nodes": sidebar_nodes,
        "globalGraph": {
            "nodes": graph_nodes,
            "links": links
        }
    }

    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

    print(f"Wrote {out_path} with {len(graph_nodes)} nodes and {len(links)} links.")

# Example:
export_global_graph("./data/recipe.db", "recipes.json")


OperationalError: no such table: Recipes